In [22]:
import os
import os.path
import numpy as np
from scipy.signal import butter, lfilter, freqz
from matplotlib import pyplot as plt
import seaborn
import csv
import re
import scipy
import yaml
import re

%matplotlib inline

In [23]:
## Making Arrays of filenames

In [24]:
os.chdir('/Users/Nush/Documents/Decision making task/AF_DMa') #change working directory

In [25]:
all_files = [] #creates a directory to store all files in
for (path, dirnames, filenames) in os.walk('/Users/Nush/Documents/Decision making task/AF_DMa'):
    if '.DS_Store' in dirnames:
        dirnames.remove('.DS_Store') #this is an apple storage folder that should not be read
    all_files.extend(os.path.join(path, name) for name in filenames if not name[0] == '.')#ignore . and .. folders
print (all_files)


['/Users/Nush/Documents/Decision making task/AF_DMa/AF_DM_DAY5_09112016/AMBIGUITY TASK 05112016_MOUSEBOTH.MPC', '/Users/Nush/Documents/Decision making task/AF_DMa/AF_DM_DAY5_09112016/AMBIGUITY TASK 05112016_MOUSENONE.MPC', '/Users/Nush/Documents/Decision making task/AF_DMa/AF_DM_DAY5_09112016/MOUSEBOTH_DAY5', '/Users/Nush/Documents/Decision making task/AF_DMa/AF_DM_DAY5_09112016/MOUSEBOTH_DAY5 copy.txt', '/Users/Nush/Documents/Decision making task/AF_DMa/AF_DM_DAY5_09112016/MouseBOTH_DAY5.tdms', '/Users/Nush/Documents/Decision making task/AF_DMa/AF_DM_DAY5_09112016/MouseBOTH_DAY5.tdms_index', '/Users/Nush/Documents/Decision making task/AF_DMa/AF_DM_DAY5_09112016/MOUSENONE_DAY5', '/Users/Nush/Documents/Decision making task/AF_DMa/AF_DM_DAY5_09112016/MouseNONE_DAY5.tdms', '/Users/Nush/Documents/Decision making task/AF_DMa/AF_DM_DAY5_09112016/MouseNONE_DAY5.tdms_index', '/Users/Nush/Documents/Decision making task/AF_DMa/AF_DM_DAY5_09112016/mousenoneday5.jpg', '/Users/Nush/Documents/Decisi

In [26]:
#this function "parse_file" reads medpc files which are files that have hidden characters and blank lines
def parse_file(filename):#defines function The heading contains def, the name of the function, parentheses, and finally a colon.
    """Parse a results file into a dictionary structure.
    
    Lines are either:
    - blank -> ignore these
    - Field name: Value -> put in the dict as a name:value entry
    - whitespace index: value value value -> continuation line for array data
    """
    results = {}#creates a dictionary
    with open(filename) as f:# create a file using open, f is the file data object we're creating for reading
        multi_line_name = None#None is frequently used to represent the absence of a value, as when default arguments are not passed to a function. 
        for line in f:
            if not line.strip():#if the line is blank then continue
                continue # Ignore this line
            if line[0] in ' \t':#escape key places a tab
                # Continuation line
                line = line.strip().split(':')[1] # Strip off the '    index:' part
                values = line.split() # Get the individual values as a list
                values = map(int, map(float, values)) # Convert all to integers
                results[multi_line_name].extend(values)
            else:#for single value entries in medpc file
                name, value = line.split(':', 1)
                name = name.strip()
                value = value.strip()
                if value:
                    # Simple one-line entry
                    results[name] = value
                else:#for text eg. subject
                    # Start of multi-line array
                    multi_line_name = name
                    results[name] = []
    return results


In [27]:
#This cell imports data from a TDMS file using the nptdms function and reads different channel data
from __future__ import print_function
from nptdms import TdmsFile
from matplotlib import pyplot as plt
#from nptdms import tdmsinfo
tdms_file = TdmsFile("/Users/Nush/Documents/Decision making task/AF_DMc/Photometry/AF039-37_DAY6bb_photometry_AF_DMc_21122016.csv")
channel = tdms_file.object('Untitled', 'Voltage_1')
data = channel.data
time = channel.time_track("Time (Voltage_1)")
plt.plot(data[0:1000]) 

KeyboardInterrupt: 

In [ ]:
#To test the parse_file function this cell reads one of these files and prints specific parts of the file and also all of its contents
contents = parse_file('/Users/Nush/Documents/Decision making task/AF_DMc/MPC/AF039_DAY6_21122016_AF_DMc')#for i in results
print (contents['Start Date'], contents['End Date'])
print(contents['Subject'])
print (contents)
#print contents['Box']

In [ ]:
#The Medpc file consists of single lines of texts e.g. subject: subject name, single values eg. 26 and arrays (m, n, u,o) following a key letter. 
#This cell attempts to read the arrays, M, N, O and U and convert them to numpy arrays. This is taking from the single file above
#The script then removes all the zero values that are inputed by the programme when no event occurs
import numpy 
M_Event=[]
N_Time=[]
M_event = numpy.array(contents['M'])
N_time = numpy.array(contents['N'])
U_Event=[]
O_Time=[]
M_event = numpy.array(contents['M'])
N_time = numpy.array(contents['N'])

for x in M_event:
    if x != 0:
        M_Event.append (x)
for y in N_time:
    if y != 0:
        N_Time.append (y)
print(M_Event, N_Time)
U_event = numpy.array(contents['U'])
O_time = numpy.array(contents['O'])
for z in U_event:
    if z != 0:
        U_Event.append (z)
for w in O_time:
    if w != 0:
        O_Time.append (w)
print(U_Event, O_Time)


In [ ]:
#This cell plots data from the arrays to check the task is working
assert len(U_Event) == len(O_Time)#checks they are the same length
for i in range (500,600):
        #print (O_time[i], U_event[i])
        plt.plot([O_time[i]], [U_event[i]],'ro')
        plt.axis([17840, 17980, 0, 20])

In [ ]:
# Subsquent cells below aim to combine different file types into one directory to analyse together

In [ ]:
#Attempt 1: Concatenating the numpty arrays from the medpc file
Medpc=[]
Medpc=np.concatenate((M_Event,N_Time,U_Event,O_Time), axis=0)
print(Medpc)


In [ ]:
#Attempt 2: This cell creates a dictionary called Medpc - in this directory I try to align the four arrays. The aim is to 
#search for an event in the array in 0 or 2 position
#and print the corresponding time in the 1 and 3 arrays in the Medpc directory. I would also like to scan for an event
#in the m array or u array and the look five seconds before and after for other events and their time stamps
Medpc={}

Medpc={'0':[M_Event],'1':[N_Time],'2':[U_Event],'3':[O_Time]}
time_events = []
for x in enumerate(Medpc['2']):
    if x == 13:
                time_events.append(x)
print (time_events)

    

#print (Medpc['2'])
events=[]
events=(Medpc.get('0','none'))
#print(events)

In [ ]:
#Attempt 3
#Assign a function: it first convert arrays to numpy arrays, then strips all the zeros then puts it all in one dictionary
def MEDPCData(analysis): 
    import numpy #library of science analysis tools
    Medpc_data={}
    M_Event=[]
    N_Time=[]
    M_event = numpy.array(contents['M'])
    N_time = numpy.array(contents['N'])
    U_Event=[]
    O_Time=[]
    M_event = numpy.array(contents['M'])
    N_time = numpy.array(contents['N'])

    for x in M_event:
        if x != 0:
            M_Event.append (x)
    for y in N_time:
        if y != 0:
            N_Time.append (y)
    print(M_Event, N_Time)
    U_event = numpy.array(contents['U'])
    O_time = numpy.array(contents['O'])
    for z in U_event:
        if z != 0:
            U_Event.append (z)
    for w in O_time:
        if w != 0:
            O_Time.append (w)
    print(U_Event, O_Time)
    Medpc={}
    Medpc={'0':[M_Event],'1':[N_Time],'2':[U_Event],'3':[O_Time]}
    
    return Medpc